<a href="https://colab.research.google.com/github/mohamedhanfi/Car-Type-Classification/blob/main/Xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

In [ ]:
from tensorflow.keras.applications import Xception

base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))


In [ ]:
for layer in base_model.layers:
    layer.trainable=False

In [ ]:
# Install the Kaggle package if not already installed
!pip install kaggle

# Download the dataset
!kaggle datasets download -d jutrera/stanford-car-dataset-by-classes-folder

# Unzip the dataset
!unzip stanford-car-dataset-by-classes-folder.zip -d car_data

Dataset URL: https://www.kaggle.com/datasets/jutrera/stanford-car-dataset-by-classes-folder
License(s): other
stanford-car-dataset-by-classes-folder.zip: Skipping, found more recently modified local copy (use --force to force download)
unzip:  cannot find or open stanford-car-dataset-by-classes-folder.zip -d car_data, stanford-car-dataset-by-classes-folder.zip -d car_data.zip or stanford-car-dataset-by-classes-folder.zip -d car_data.ZIP.


In [ ]:
!unzip stanford-car-dataset-by-classes-folder.zip -d car_data

Archive:  stanford-car-dataset-by-classes-folder.zip
replace car_data/anno_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# Define the paths
train_dir = '/content/car_data/car_data/car_data/train'
val_dir = '/content/car_data/car_data/car_data/test'

# Data Augmentation for training
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Data Generator for validation
val_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )




In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model

# Create data generators
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

# Load pre-trained Xception model
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Unfreeze more layers for fine-tuning
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Add custom layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
outputs = Dense(196, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=outputs)


Found 8144 images belonging to 196 classes.
Found 8041 images belonging to 196 classes.


In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)


In [ ]:
# Train the model
history = model.fit(train_data, epochs=10, validation_data=val_data, callbacks=[early_stopping, checkpoint, reduce_lr])

Epoch 1/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 371s 1s/step - accuracy: 0.0054 - loss: 18.8912 - val_accuracy: 0.0057 - val_loss: 16.3705 - learning_rate: 1.0000e-04
Epoch 2/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 303s 1s/step - accuracy: 0.0087 - loss: 15.5117 - val_accuracy: 0.0085 - val_loss: 12.9441 - learning_rate: 1.0000e-04
Epoch 3/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 301s 1s/step - accuracy: 0.0153 - loss: 12.1180 - val_accuracy: 0.0634 - val_loss: 9.9453 - learning_rate: 1.0000e-04
Epoch 4/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 300s 1s/step - accuracy: 0.1865 - loss: 8.3708 - val_accuracy: 0.4083 - val_loss: 6.4479 - learning_rate: 1.0000e-04
Epoch 5/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 323s 1s/step - accuracy: 0.5005 - loss: 5.6822 - val_accuracy: 0.5711 - val_loss: 4.8315 - learning_rate: 1.0000e-04
Epoch 6/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 337s 1s/step - accuracy: 0.6739 - loss: 4.2788 - val_accuracy: 0.6726 - val_loss: 3.9159 - learning_rate: 1.0000e-04
Epoch 7/10
255/255 ━━━━━━━━━━━━━━━━━━━━ 331s 1s/step 